<a href="https://colab.research.google.com/github/debg48/lung-cancer-detection/blob/main/Lung_Cancer_Detection_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opendatasets --upgrade --quiet

In [2]:
dataset_url = 'https://www.kaggle.com/datasets/mohamedhanyyy/chest-ctscan-images'

In [3]:
import opendatasets as od
od.download(dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: debg48
Your Kaggle Key: ··········


100%|██████████| 119M/119M [00:06<00:00, 18.8MB/s]


In [4]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np

In [5]:
from PIL import Image 


img = Image.open ('/content/chest-ctscan-images/Data/train/large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa/000003 (3).png')
img.size

(437, 258)

In [6]:
datagen=ImageDataGenerator(
                           rescale=1./255.,
                           validation_split=0.15,
                           zoom_range=0.1,
                           horizontal_flip=True,
                           fill_mode='nearest'                           
                        )

In [7]:
train_generator=datagen.flow_from_directory(
                    directory="/content/chest-ctscan-images/Data/train",
                    subset="training",
                    color_mode='rgb',
                    class_mode='categorical',
                    batch_size=32,
                    shuffle=True,
                    seed=0,
                    target_size=(240,240)
              )

valid_generator=datagen.flow_from_directory(
                    directory="/content/chest-ctscan-images/Data/valid",
                    subset="validation",
                    color_mode='rgb',
                    class_mode='categorical',
                    batch_size=32,
                    shuffle=True,
                    seed=0,
                    target_size=(240,240)
              )

Found 522 images belonging to 4 classes.
Found 9 images belonging to 4 classes.


In [8]:
batch_size = 16
num_classes = 4

In [9]:
def create_resnet_model():
    pretrained_model = tf.keras.applications.ResNet50V2(include_top=False , weights='imagenet')
    pretrained_model.trainable = True
    model = tf.keras.Sequential([
        pretrained_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(64, activation = 'relu'),
        tf.keras.layers.Dense(num_classes, activation = 'softmax')
    ])
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model

In [10]:
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint

In [11]:
LR_START = 1.0e-5
LR_MAX = 2.5e-5 
LR_MIN = LR_START
LR_RAMPUP_EPOCHS = 1
LR_SUSTAIN_EPOCHS = 0 # 3
LR_EXP_DECAY = 0.80

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = LR_START + (epoch * (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS)
    elif epoch < (LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS):
        lr = LR_MAX
    else:
        lr = LR_MIN + (LR_MAX - LR_MIN) * LR_EXP_DECAY ** (epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS)
#    print('For epoch', epoch, 'setting lr to', lr)
    return lr

    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)

checkpoint = ModelCheckpoint(filepath='best_weights.hdf5',
                             save_weights_only=True,
                             monitor='val_accuracy',
                             mode='max',
                             save_best_only=True)

In [12]:
model_res = create_resnet_model()

94668760/94668760 [==============================] - 5s 0us/step


In [13]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
#STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
hist_incep = model_res.fit(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5,
                    callbacks = [lr_callback, checkpoint]
)


Epoch 1: LearningRateScheduler setting learning rate to 1e-05.
Epoch 1/5
16/16 [==============================] - ETA: 0s - loss: 1.3307 - accuracy: 0.3735

16/16 [==============================] - 60s 705ms/step - loss: 1.3307 - accuracy: 0.3735 - lr: 1.0000e-05

Epoch 2: LearningRateScheduler setting learning rate to 2.5e-05.
Epoch 2/5
16/16 [==============================] - ETA: 0s - loss: 0.9926 - accuracy: 0.6245

16/16 [==============================] - 9s 562ms/step - loss: 0.9926 - accuracy: 0.6245 - lr: 2.5000e-05

Epoch 3: LearningRateScheduler setting learning rate to 2.2e-05.
Epoch 3/5
16/16 [==============================] - ETA: 0s - loss: 0.7456 - accuracy: 0.7388

16/16 [==============================] - 9s 567ms/step - loss: 0.7456 - accuracy: 0.7388 - lr: 2.2000e-05

Epoch 4: LearningRateScheduler setting learning rate to 1.9600000000000002e-05.
Epoch 4/5
16/16 [==============================] - ETA: 0s - loss: 0.6050 - accuracy: 0.8184

16/16 [==============================] - 10s 570ms/step - loss: 0.6050 - accuracy: 0.8184 - lr: 1.9600e-05

Epoch 5: LearningRateScheduler setting learning rate to 1.7680000000000004e-05.
Epoch 5/5
16/16 [==============================] - ETA: 0s - loss: 0.4902 - accuracy: 0.8673

16/16 [==============================] - 11s 659ms/step - loss: 0.4902 - accuracy: 0.8673 - lr: 1.7680e-05
